# GeoJSON ファイルのロードと計算処理の分散実行

このエクササイズでは、**行ごとにポリゴン情報が含まれる** [GeoJSONSeq](https://gdal.org/drivers/vector/geojsonseq.html) ファイルを Spark でロードし、各ワーカー上で分散処理を行います。

Spark SQL は自動的にJSONデータセットのスキーマを推測しデータフレームとしてロードすることができます。この変換は、JSONファイル上のSparkSession.read.jsonを使って行うことができます。

`spark.read.json` が求めるファイルは一般的な JSON ファイルではないことに注意してください。各行は別個の自己内包の有効な JSON オブジェクトでなければなりません。これは [JSON Lines](https://jsonlines.org/) や [Newline-Delimited JSON](http://ndjson.org/) とも呼ばれます。

http://mogile.web.fc2.com/spark/sql-data-sources-json.html

## JSON ファイルのロード

In [0]:
%%time

df = spark.read.json("/mnt/testblob2/geojson/merged_nd.geojsonl.json")

CPU times: user 36.1 ms, sys: 18 ms, total: 54.1 ms
Wall time: 13.4 s

In [0]:
display(df)

geometry,properties,type
"List(List(List(List(139.2514364, 37.9174405), List(139.2518658, 37.9176022), List(139.2515764, 37.9178179), List(139.2515616, 37.9178094), List(139.2515459, 37.9178175), List(139.2512581, 37.9175647), List(139.2514364, 37.9174405))), Polygon)","List(2020, 2022, 100, e17bca66-87cf-4d75-a357-dcec49a17d48, 2022_151017, 151017, /Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017, 37.917619139, 139.251546719, ccccbc5d-3b2c-4a06-9d8b-aef5b930daf9)",Feature
"List(List(List(List(139.2318403, 37.8640836), List(139.232175, 37.8642189), List(139.2323779, 37.8643338), List(139.2317783, 37.8642065), List(139.2318403, 37.8640836))), Polygon)","List(2020, 2022, 200, 218aac09-72f3-4d7b-9459-e6c275330465, 2022_151017, 151017, /Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017, 37.864208476, 139.232014575, 1acc7fb0-9c51-4dc6-ae4c-9e63cbaa3632)",Feature
"List(List(List(List(139.1877035, 37.8626389), List(139.1876241, 37.8623294), List(139.1878656, 37.8623795), List(139.1877831, 37.8626259), List(139.1877035, 37.8626389))), Polygon)","List(2020, 2022, 100, 3cb10bab-5a07-4745-b0ee-a0fa9c46a2b0, 2022_151017, 151017, /Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017, 37.862469773, 139.187742061, 82e377ec-f41c-4fc6-b8e2-f561fc71d64a)",Feature
"List(List(List(List(139.2270315, 37.9552805), List(139.2272117, 37.9553614), List(139.2270723, 37.9555833), List(139.2269022, 37.9555181), List(139.2270315, 37.9552805))), Polygon)","List(2020, 2022, 200, 9bb4045d-deb5-437f-b01f-a607cdd19d7e, 2022_151017, 151017, /Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017, 37.955433882, 139.227054847, f9b74f60-c6fc-4d43-80c5-5e5e4110315c)",Feature
"List(List(List(List(139.1974306, 37.9342118), List(139.197443, 37.9342136), List(139.1981092, 37.9345739), List(139.1979453, 37.9347665), List(139.1979058, 37.9347524), List(139.197312, 37.9344327), List(139.1972971, 37.9344069), List(139.1973069, 37.934364), List(139.1974306, 37.9342118))), Polygon)","List(2020, 2022, 100, 73c08a4e-9816-4f44-b28c-8df7d53da99d, 2022_151017, 151017, /Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017, 37.934490886, 139.197689946, cc94d24c-e029-4bdf-9df3-9e51667b4648)",Feature
"List(List(List(List(139.192043, 37.9323869), List(139.1918731, 37.932544), List(139.1913201, 37.9321821), List(139.1913179, 37.9321556), List(139.1913157, 37.9321291), List(139.1914777, 37.9319721), List(139.1920027, 37.9322967), List(139.1919695, 37.9323234), List(139.192043, 37.9323869))), Polygon)","List(2020, 2022, 100, 73435e88-26db-4960-b12c-7ba2c3540cef, 2022_151017, 151017, /Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017, 37.932256085, 139.191676797, 09258d7b-9273-4246-8ac7-d4b10fb5a8ea)",Feature
"List(List(List(List(139.1988774, 37.9361606), List(139.198868, 37.9361679), List(139.1982349, 37.9358337), List(139.1982331, 37.9358073), List(139.1982234, 37.935796), List(139.1983584, 37.9356381), List(139.1983775, 37.9356456), List(139.1984072, 37.9356381), List(139.1990073, 37.9359623), List(139.1990123, 37.9359876), List(139.1988774, 37.9361606))), Polygon)","List(2020, 2022, 100, 18b1e658-74ba-4305-9a49-d461bee8ec02, 2022_151017, 151017, /Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017, 37.935895161, 139.198618329, 55bea87f-9b10-4ec5-9623-8a412f4cad36)",Feature
"List(List(List(List(139.1928758, 37.9349333), List(139.1922443, 37.9345082), List(139.1924273, 37.9343364), List(139.1930569, 37.9347627), List(139.1928758, 37.9349333))), Polygon)","List(2020, 2022, 100, 194710d2-8f88-4485-894d-5d20561b689a, 2022_151017, 151017, /Users/han/work/Space/geojson/2022_151017.json|layername=2022_151017, 37.934634864, 139.192650631, a18b458b-c625-4b56-8fe2-ff8e470bc7fc)",Feature
"List(List(List(List(139.203721, 37.935807), List(139.2037639, 37.9355634), List(139.2045509, 37.9356632), List(139.2045711, 37.9357161), List(139.2045431, 37.935877), List(139.2045274, 37.9358864), L

In [0]:
# RDD のパーティション数を返します
df.rdd.getNumPartitions()

Out[3]: 8

## キャッシュ
Pyspark の `cache()` メソッドを使用して変換の中間結果をキャッシュし、キャッシュされた上で実行される他の変換がより高速に実行されるようにします。変換の結果をキャッシュすることは、長時間実行される PySpark アプリケーション/ジョブのパフォーマンスを向上させるための最適化のトリックの 1 つです。

https://sparkbyexamples.com/pyspark/pyspark-cache-explained/

In [0]:
df.cache()

## ユーザー定義関数 (UDF) 
ユーザー定義関数 (UDF) は、ユーザーによって定義された関数であり、ユーザー環境でカスタム ロジックを再利用できます。Databricks は、拡張可能なロジックを配布できるように、さまざまな種類の UDF をサポートしています。<br><br>

- https://docs.databricks.com/udf/index.html
- https://docs.databricks.com/udf/python.html#use-udf-with-dataframes

### Polygon 座標から面積を計算する UDF を定義

GeoJson から得られる Polygon 座標を投影座標に変換した後、m^2 の単位で面積を計算します。

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, DecimalType
from pyspark.sql.functions import lit, pandas_udf, PandasUDFType, col
from shapely.geometry import Polygon, shape
import geopandas as gpd
import pyproj
from shapely.ops import transform

wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:3100')
project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform

def calc_area(geometry): 
    polygon_geom = shape(geometry.asDict(True))
    utm_point = transform(project, polygon_geom)
    area = str(utm_point.area)
  
    return area

calc_area_udf = udf(calc_area, StringType())

In [0]:
%%time

# area 列を追加
joined = df.withColumn("area", calc_area_udf(col("geometry")))


CPU times: user 4.26 ms, sys: 91 µs, total: 4.35 ms
Wall time: 85.1 ms

## ファイルに保存
Spark の遅延評価のため、下記の保存を実行するまで実際の演算は実行されません。

In [0]:
%%time
joined.write.mode('Overwrite').json("/mnt/testblob2/geojson/sparkarea")

CPU times: user 378 ms, sys: 160 ms, total: 538 ms
Wall time: 2min 5s

## 参考　非並列ロード
注意：読み込みには約 4 分程度かかります。

In [0]:
from geopandas import GeoDataFrame

# Loading cropfields Data
geo_df = GeoDataFrame.from_file('/dbfs/mnt/testblob2/geojson/merged_nd.geojsonl.json')
geo_df.head(5)

Out[14]:

,polygon_uuid,land_type,issue_year,edit_year,last_polygon_uuid,local_government_cd,point_lng,point_lat,layer,path,geometry
0,ccccbc5d-3b2c-4a06-9d8b-aef5b930daf9,100,2022,2020,e17bca66-87cf-4d75-a357-dcec49a17d48,151017,139.251547,37.917619,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((139.25144 37.91744, 139.25187 37.917..."
1,1acc7fb0-9c51-4dc6-ae4c-9e63cbaa3632,200,2022,2020,218aac09-72f3-4d7b-9459-e6c275330465,151017,139.232015,37.864208,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((139.23184 37.86408, 139.23218 37.864..."
2,82e377ec-f41c-4fc6-b8e2-f561fc71d64a,100,2022,2020,3cb10bab-5a07-4745-b0ee-a0fa9c46a2b0,151017,139.187742,37.862470,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((139.18770 37.86264, 139.18762 37.862..."
3,f9b74f60-c6fc-4d43-80c5-5e5e4110315c,200,2022,2020,9bb4045d-deb5-437f-b01f-a607cdd19d7e,151017,139.227055,37.955434,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((139.22703 37.95528, 139.22721 37.955..."
4,cc94d24c-e029-4bdf-9df3-9e51667b4648,100,2022,2020,73c08a4e-9816-4f44-b28c-8df7d53da99d,151017,139.197690,37.934491,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((139.19743 37.93421, 139.19744 37.934..."
...,...,...,...,...,...,...,...,...,...,...,...
1312839,305fc4cb-e67f-47ea-85e8-5571d25c073b,200,2022,2021,None,155861,139.251471,38.462826,2022_155861,/Users/han/work/Space/geojson/2022_155861.json...,"POLYGON ((139.25169 38.46288, 139.25165 38.462..."
1312840,3b3eea79-fe46-47bb-a18d-d87bf6a2b5e5,200,2022,2021,None,155861,139.251366,38.462700,2022_155861,/Users/han/work/Space/geojson/2022_155861.json...,"POLYGON ((139.25168 38.46267, 139.25152 38.462..."
1312841,43c0db6d-fc49-4f53-8c45-187a5ffee4b9,200,2022,2021,None,155861,139.251611,38.460512,2022_155861,/Users/han/work/Space/geojson/2022_155861.json...,"POLYGON ((139.25147 38.46058, 139.25148 38.460..."
1312842,b27ac6b2-a942-4b1b-9261-8c99b2944589,200,2022,2021,None,155861,139.260146,38.487099,2022_155861,/Users/han/work/Space/geojson/2022_155861.json...,"POLYGON ((139.26003 38.48724, 139.25997 38.487..."


In [0]:
# CRS の確認
geo_df.crs

Out[15]: <Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [0]:
# CRS の変換
geo_df = geo_df.to_crs(epsg=3100)
geo_df.head(5)

Out[17]:

,polygon_uuid,land_type,issue_year,edit_year,last_polygon_uuid,local_government_cd,point_lng,point_lat,layer,path,geometry
0,ccccbc5d-3b2c-4a06-9d8b-aef5b930daf9,100,2022,2020,e17bca66-87cf-4d75-a357-dcec49a17d48,151017,139.251547,37.917619,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((346302.941 4198096.422, 346341.024 4..."
1,1acc7fb0-9c51-4dc6-ae4c-9e63cbaa3632,200,2022,2020,218aac09-72f3-4d7b-9459-e6c275330465,151017,139.232015,37.864208,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((344468.078 4192208.272, 344497.806 4..."
2,82e377ec-f41c-4fc6-b8e2-f561fc71d64a,100,2022,2020,3cb10bab-5a07-4745-b0ee-a0fa9c46a2b0,151017,139.187742,37.862470,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((340582.259 4192122.451, 340574.607 4..."
3,f9b74f60-c6fc-4d43-80c5-5e5e4110315c,200,2022,2020,9bb4045d-deb5-437f-b01f-a607cdd19d7e,151017,139.227055,37.955434,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((344237.511 4202335.827, 344253.514 4..."
4,cc94d24c-e029-4bdf-9df3-9e51667b4648,100,2022,2020,73c08a4e-9816-4f44-b28c-8df7d53da99d,151017,139.197690,37.934491,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((341591.500 4200047.875, 341592.594 4..."


In [0]:
# GeoPandas の面積計算(m^2)
geo_df["area"] = geo_df['geometry'].area
geo_df.head(5)

Out[18]:

,polygon_uuid,land_type,issue_year,edit_year,last_polygon_uuid,local_government_cd,point_lng,point_lat,layer,path,geometry,area
0,ccccbc5d-3b2c-4a06-9d8b-aef5b930daf9,100,2022,2020,e17bca66-87cf-4d75-a357-dcec49a17d48,151017,139.251547,37.917619,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((346302.941 4198096.422, 346341.024 4...",1129.153797
1,1acc7fb0-9c51-4dc6-ae4c-9e63cbaa3632,200,2022,2020,218aac09-72f3-4d7b-9459-e6c275330465,151017,139.232015,37.864208,2022_151017,/Users/han/work/Space/geojson/2022_151017.json...,"POLYGON ((344468.078 4192208.272, 344497.806 4...",452.054343


注意：保存には約 12 分程度かかります。

In [0]:
%%time
geo_df.to_file("/dbfs/mnt/testblob2/geojson/minamiuonuma_area.json", driver='GeoJSONSeq')

CPU times: user 10min 26s, sys: 35.2 s, total: 11min 1s
Wall time: 11min 36s